In [1]:
import sys
sys.path.append("/home/yamato/Application/reduction_tools/")
sys.path.append("/home/yamato/Application/keplerian_mask/")
import reduction_utils_mpi as utils
from keplerian_mask import make_mask
import casatasks
from linedictionary import line_dict
from diskdictionary import disk_dict
import shutil
import os
import analysis_utils as au
import matplotlib.pyplot as plt
import imaging_utils as iutils
import numpy as np

%matplotlib widget

source = "MWC_480"
msdir = "/works/yamato/C2D_disk/v1_product/measurement_set/"
vispath = "./data/measurement_set/"
impath = "./data/image/"
ms_list = [msdir + f"{source}_EB{i+1}_continuum.ms" for i in range(5)]

You appear to be importing analysisUtils into python (not CASA). version =  3.10.5
CASAPATH is not defined, so I am skipping a lot of imports
Successfully imported `make_mask`.


In [7]:
# combine all continuum measurement sets
concatvis = vispath + f"{source}_continuum.ms"
casatasks.concat(vis=ms_list, concatvis=concatvis)

In [3]:
# CLEAN
vis = vispath + f"{source}_continuum.ms"
imagename = impath + f"{source}_continuum_tapered"
cellsize = "0.02arcsec"
imsize = utils.get_imsize(
	vis=vis,
	field=source,
	cellsize=cellsize,
)
scales = [0, 10, 30]
weighting = "superuniform"
deconvolver = "mtmfs"
nterms = 2
npixels = 8
target_bmaj = 0.3
incl = disk_dict[source]["incl"]
PA = disk_dict[source]["PA"]

# elliptical mask
mask_semimaj = 2.3
disk_mask = "ellipse[[{:s}, {:s}], [{:.1f}arcsec, {:.1f}arcsec], {:.1f}deg]".format(
	"04h58m46.27936s", "+29d50m36.398586s", mask_semimaj, mask_semimaj * np.cos(np.deg2rad(incl)), PA
)

# get uvtaper parameter to achieve the desired beam
uvtaper = iutils.calc_uvtaper(
    vis=vis,
    imagename=imagename,
    field="MWC_480",
    specmode="mfs",
    weighting=weighting,
    npixels=npixels,
    cellsize=cellsize,
    target_beam=(target_bmaj, target_bmaj * np.cos(np.deg2rad(incl)), PA),
)

rms = utils.calc_sensitivity(vis, cellsize=cellsize, imsize=imsize, weighting=weighting, uvtaper=uvtaper, npixels=npixels)

for ext in [".alpha*", ".image*", ".mask", ".model*", ".pb*", ".psf*", ".residual*", ".sumwt*"]:
    os.system("rm -r " + imagename + ext)
    
casatasks.tclean(
    vis=vis,
    imagename=imagename,
    imsize=imsize,
    cell=cellsize,
    specmode="mfs",
    deconvolver=deconvolver,
    scales=scales,
    weighting=weighting,
    npixels=npixels,
    nterms=nterms,
    niter=100000,
    threshold=f"{rms*3}Jy",
    usemask="user",
    mask=disk_mask,
    uvtaper=uvtaper,
    # nsigma=3.0
)

Starting to estimate the uvtaper which achieve the target beam of 0.3000 arcsec x 0.2396 arcsec (P.A. = 148.0000 deg)
Pixel size: 0.02arcsec
Image size: 1728
Calculating and fitting PSF with the original weighting scheme (weighting = superuniform)



0%....10....20....30....40....50....60....70....80....90....100%


Done.
Restoring beam shape: 0.1815 arcsec x 0.1192 arcsec (P.A. = 0.0534 deg)
Windowing out the central region for computational efficiency
Image size of windowed psf: 600
Calculating uvtaper parameter
Done. Best-fit uvtaper parameter: ['0.286846593396618arcsec', '0.23043151604983847arcsec', '141.0040513435651deg']
Calculating the resulting beam shape after uvtaper



0%....10....20....30....40....50....60....70....80....90....100%


Done.
Restoring beam shape after uvtaper: 0.3000 arcsec x 0.2396 arcsec (P.A. = -32.0099 deg)
JvM epsilon: 1.0005
Theoretical estimates of image sensitivity
# ./data/measurement_set/MWC_480_continuum.ms
# specmode = 'mfs'
# spw = ''
# uvtaper = [0.286846593396618arcsec, 0.23043151604983847arcsec, 141.0040513435651deg]
(True, 1.624040334381604e-05, 1.3985476861872201)
# Briggs sensitivity with robust = 0.5: 1.624040334381604e-05 Jy
# Relative to natural weighting: 1.3985476861872201
### Averaged sensitivity over 1 measurement sets: 1.624040334381604e-05 Jy



0%....10....20....30....40....50....60....70....80....90....100%

0%....10....20....30....40....50....60....70....80....90....100%

0%....10....20....30....40....50....60....70....80....90....100%

0%....10....20....30....40....50....60....70....80....90....100%

0%....10....20....30....40....50....60....70....80....90....100%

0%....10....20....30....40....50....60....70....80....90....100%


{'cleanstate': 'running',
 'cyclefactor': 1.0,
 'cycleiterdone': 0,
 'cycleniter': 100000,
 'cyclethreshold': 2.435892156427144e-06,
 'interactiveiterdone': 5490,
 'interactivemode': False,
 'interactiveniter': 0,
 'interactivethreshold': 0.0,
 'iterdone': 5490,
 'loopgain': 0.10000000149011612,
 'maxpsffraction': 0.800000011920929,
 'maxpsfsidelobe': 0.04470716416835785,
 'minpsffraction': 0.05000000074505806,
 'niter': 100000,
 'nmajordone': 5,
 'nsigma': 0.0,
 'stopcode': 2,
 'summarymajor': array([   0,   55, 1224, 5224, 5490]),
 'summaryminor': {0: {0: {0: {'iterDone': [55.0, 1169.0, 4000.0, 266.0],
     'peakRes': [0.007106215693056583,
      0.00035478500649333,
      0.0004624586727004498,
      4.871765850111842e-05],
     'modelFlux': [0.44499167799949646,
      0.5347276329994202,
      0.5394614934921265,
      0.539538562297821],
     'cycleThresh': [0.007184753660112619,
      0.00035531038884073496,
      4.872120916843414e-05,
      4.872120916843414e-05]}}}},
 'thresho

In [5]:
casatasks.imsmooth(
    imagename=imagename + ".image.tt0",
    major="0.2arcsec",
    minor="0.2arcsec",
    pa="0deg",
    targetres=True,
    outfile=imagename + ".image.tt0.imsmooth",
)

In [2]:
casatasks.exportfits(
    imagename="./data/image/MWC_480_continuum_tapered.image.tt0/",
    fitsimage="./data/image/MWC_480_continuum_tapered.image.tt0.fits",
    dropstokes=True,
    overwrite=True,
)